# Cognyx client debug notebook

This notebook is used to debug the Cognyx client used in cognyx automations.
you will need a .env file with a JWT token to run this notebook. You can get this after logging into the cognyx app.

In the cell below, define the bom_id and view_id you want to use (view_id will default to default view if not provided)

In [1]:
import os

jwt = os.getenv("COGNYX_TOKEN")

bom_id = "5a012f0d-3215-429b-bef9-dd1be54d9da2"
view_id = None
base_url = "http://cognyx-backend.test"

if not bom_id:
    raise ValueError("bom_id is required")

It will load the bom data and return it.

In [2]:
from cognyx_bom_sdk.cognyx import CognyxClient

cognyx = CognyxClient(base_url=base_url, jwt_token=jwt)

bom_data = await cognyx.load_bom_data(bom_id, view_id)

When the bom is loaded it will then initialize the BomClient

In [3]:
from cognyx_bom_sdk.client import BomClient


def update_callback(update: dict) -> None:
    """Callback function to handle updates to the BOM."""
    print("Update:", update)


client = BomClient(bom_data=bom_data, update_callback=update_callback)

You can now use the client to interact with the BOM.

In [4]:
from functools import reduce

from cognyx_bom_sdk.models import BomInstance

mass_attribute = "added mass g"


def get_instance_mass(instance: BomInstance) -> int:
    """Get the mass of a BOM instance.

    Args:
        instance: BOM instance

    Returns:
        The mass of the instance
    """
    instance_mass = client.get_instance_attribute(instance.id, mass_attribute)

    if instance_mass is None:
        return 0

    print(f"instance {instance.name} weighs {instance_mass}g")
    return instance.quantity * instance_mass


reduce(lambda total, instance: total + get_instance_mass(instance), client.list_bom_instances(), 0)

0

In [5]:
instance = client.get_bom_instance("branch a")
attribute = client.get_instance_attribute(instance.id, "country")

result = await cognyx.update_instance(instance.id, {"quantity": 3})


attribute_change = await cognyx.update_instance_attribute(
    instance.id, "c782de11-0d87-4f07-91f3-9461edbbf3d2", "India"
)

attribute_value = None
for attr in attribute_change.custom_attributes:
    if attr.name.lower() == "country":
        attribute_value = attr.value
        break

attribute_value

'India'